# Gain Statistical Insights into Your Data

Woodwork provides methods on your DataFrames to allow you to use the typing information stored by Woodwork to better understand your data.

Follow along to learn how to use Woodwork's statistical methods on a DataFrame of retail data while demonstrating the full capabilities of the functions.

In [ ]:
import pandas as pd
import numpy as np
from woodwork.demo import load_retail

df = load_retail()
df.ww

## DataFrame.ww.describe

Use `df.ww.describe()` to calculate statistics for the columns in a DataFrame, returning the results in the format of a pandas DataFrame with the relevant calculations done for each column. Note, that both `nan` and `(nan, nan)` values contribute to `nan_count` for LatLong logical types

In [ ]:
df.ww.describe()

There are a couple things to note in the above dataframe:

- The Woodwork index, `order_product_id`, is not included
- We provide each column's typing information according to Woodwork's typing system
- Any statistics that can't be calculated for a column, such as `num_false` on a `Datetime` are filled with `NaN`.
- Null values do not get counted in any of the calculations other than `nunique`

## DataFrame.ww.value_counts

Use `df.ww.value_counts()` to calculate the most frequent values for each column that has `category` as a standard tag. This returns a dictionary where each column is associated with a sorted list of dictionaries. Each dictionary contains `value` and `count`.

In [ ]:
df.ww.value_counts()

## DataFrame.ww.dependence

`df.ww.dependence` calculates several dependence/correlation measures between all pairs of relevant columns. Certain types, like strings, can't have dependence calculated.

The mutual information between columns `A` and `B` can be understood as the amount of knowledge you can have about column `A` if you have the values of column `B`. The more mutual information there is between `A` and `B`, the less uncertainty there is in `A` knowing `B`, and vice versa. 

The Pearson correlation coefficient measures the linear correlation between `A` and `B`.

In [ ]:
df.ww.dependence(measure="all", nrows=10000)

#### Available Parameters
`df.ww.dependence` provides various parameters for tuning the dependence calculation.

- `measure` - Which dependence measures to calculate. A list of measures can be provided to calculate multiple measures at once.  Valid measure strings:
    - "pearson": calculates the Pearson correlation coefficient
    - "mutual": calculates the mutual information between columns
    - "max": calculates both Pearson and mutual information and
        returns max(abs(pearson), mutual) for each pair of columns
    - "all": includes columns for "pearson", "mutual", and "max"
- `num_bins` - In order to calculate mutual information on continuous data, Woodwork bins numeric data into categories. This parameter allows you to choose the number of bins with which to categorize data.
    - Defaults to using 10 bins
    - The more bins there are, the more variety a column will have. The number of bins used should accurately portray the spread of the data.
- `nrows` - If `nrows` is set at a value below the number of rows in the DataFrame, that number of rows is randomly sampled from the underlying data
    - Defaults to using all the available rows.
    - Decreasing the number of rows can speed up the mutual information calculation on a DataFrame with many rows, but you should be careful that the number being sampled is large enough to accurately portray the data.
- `include_index` - If set to `True` and an index is defined with a logical type that is valid for mutual information, the index column will be included in the mutual information output.
    - Defaults to `False`

Now that you understand the parameters, you can explore changing the number of bins. Note—this only affects numeric columns `quantity` and `unit_price`. Increase the number of bins from 10 to 50, only showing the impacted columns.

In [ ]:
dep_df = df.ww.dependence(measure="all", nrows=10000)
dep_df[dep_df['column_1'].isin(['unit_price', 'quantity']) | dep_df['column_2'].isin(['unit_price', 'quantity'])]

In [ ]:
dep_df = df.ww.dependence(measure="all", nrows=10000, num_bins=50)
dep_df[dep_df['column_1'].isin(['unit_price', 'quantity']) | dep_df['column_2'].isin(['unit_price', 'quantity'])]

In order to include the index column in the mutual information output, run the calculation with `include_index=True`.

In [ ]:
dep_df = df.ww.dependence(measure="all", nrows=10000, num_bins=50, include_index=True)
dep_df[dep_df['column_1'].isin(['order_product_id']) | dep_df['column_2'].isin(['order_product_id'])]

## Outlier Detection with Series.ww.box_plot_dict

Woodwork allows for univariate outlier detection using the IQR, or interquartile range, method. This can be done on a by-column basis using the `series.ww.box_plot_dict` method that identifies outliers and includes the statistical data necessary for building a box and whisker plot.

In [ ]:
total = df.ww['total']
box_plot_dict = total.ww.box_plot_dict()

print('high bound: ', box_plot_dict['high_bound'])
print('low_bound: ', box_plot_dict['low_bound'])
print('quantiles: ', box_plot_dict['quantiles'])
print('number of low outliers: ', len(box_plot_dict['low_values']))
print('number of high outliers: ', len(box_plot_dict['high_values']))

We can see that the `total` column in the retail dataset has many outliers, and they are skewed more towards the top of the dataset. There are around 400K rows in the dataframe, so about 8% of all values are outliers. Let's also look at a normally distributed column of data of the same length and see what the statistics generated for it look like.

In [ ]:
rnd = np.random.RandomState(33)
s = pd.Series(rnd.normal(50, 10, 401604))
s.ww.init()
box_plot_dict = s.ww.box_plot_dict() 
print('high bound: ', box_plot_dict['high_bound'])
print('low_bound: ', box_plot_dict['low_bound'])
print('quantiles: ', box_plot_dict['quantiles'])
print('number of low outliers: ', len(box_plot_dict['low_values']))
print('number of high outliers: ', len(box_plot_dict['high_values']))

With the normally distributed set of data, the likelyhood of outliers is closer to what we'd expect, around .7%. 